In [ ]:
import os 
import serial
import time
import csv
import numpy as np
import matplotlib.pyplot as plt
from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError

# Plate type 
plate_types = {
    "Thinfilm":    {"offsetX": 17.55, "offsetY": 14.55, "colDist": 30.95, "rowDist": 27.95, "rows": 3, "cols": 4},
    "Corning_24":  {"offsetX": 17.48, "offsetY": 13.80, "colDist": 19.30, "rowDist": 19.30, "rows": 4, "cols": 6},
    "Corning_48":  {"offsetX": 18.16, "offsetY": 10.12, "colDist": 13.08, "rowDist": 13.08, "rows": 6, "cols": 8},
    "Corning_96":  {"offsetX": 14.38, "offsetY": 11.23, "colDist": 9.00,  "rowDist": 9.00,  "rows": 8, "cols": 12},
}

# Collect reference spectrum
def collect_ref(device, well):
    print(f" Collecting reference measurement for well {well}...")

    folder_path = 'dark_ref_data'
    os.makedirs(folder_path, exist_ok=True)

    try:
        # Get and calibrate spectrum
        spec = np.array(device.get_formatted_spectrum())
        wl = device.get_wavelengths()
        reference = spec

        # Save to CSV
        filename = os.path.join(folder_path, f"{well}ref.csv")
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Wavelength (nm)", "Reference Reflectance"])
            for w, r in zip(wl, reference):
                writer.writerow([w, r])

        print(f"Saved {filename}")
        return {well: reference}

    except OceanDirectError as e:
        print(f"Failed at {well}: {e.get_error_details()}")
        return {}

# Collect dark spectrum
def collect_dark(device, well):
    print(f" Collecting dark measurement for well {well}...")

    folder_path = 'dark_ref_data'
    os.makedirs(folder_path, exist_ok=True)

    try:
        # Get and calibrate spectrum
        spec = np.array(device.get_formatted_spectrum())
        wl = device.get_wavelengths()
        dark = spec

        # Save to CSV
        filename = os.path.join(folder_path, f"{well}dark.csv")
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Wavelength (nm)", "Dark Reflectance"])
            for w, r in zip(wl, dark):
                writer.writerow([w, r])

        print(f"Saved {filename}")
        return {well: dark}

    except OceanDirectError as e:
        print(f"Failed at {well}: {e.get_error_details()}")
        return {}
    
# Kamome and OceanDirect scan dark reference spectrum 
def scan_dark(port, plate_type_name, wait_interval, start_well, end_well, device):
    import queue

    input("Please put an empty plate onto Kamome, and turn off the light source. Press Enter when ready")
    well_queue = queue.Queue()

    plate = plate_types.get(plate_type_name)
    if not plate:
        print("Invalid plate type")
        return None, None

    # Prepare the scan command
    command = f"scan_select,{plate['offsetX']},{plate['offsetY']},{plate['colDist']},{plate['rowDist']},{plate['rows']},{plate['cols']},{wait_interval},{start_well},{end_well}\n"

    # Open serial communication
    ser = serial.Serial(port, 9600, timeout=2)
    time.sleep(0.5)
    ser.write(command.encode())

    print(f"Sent: {command.strip()}")
    print("Listening to Arduino")

    time.sleep(0.1)

    # Listen for responses from Arduino
    while True:
        line = ser.readline().decode().strip()
        if line:
            print("Arduino:", line, end='\r')  # Overwrites the current line
            if line.startswith("Well: "):
                well = line.replace("Well: ", "")
                well_queue.put(well)
                print(f"Well: {well} ", end='', flush=True)  # Append to the same line
                
                # Collect and save the measurement for the current well
                collect_dark(device, well)

            elif "Scan completed" in line:
                break

    ser.close()
    
# Kamome and OceanDirect scan reference spectrum
def scan_ref(port, plate_type_name, wait_interval, start_well, end_well, device):
    import queue
    input("Please turn on the light source. Press Enter when ready")
    well_queue = queue.Queue()

    plate = plate_types.get(plate_type_name)
    if not plate:
        print("Invalid plate type")
        return None, None

    # Prepare the scan command
    command = f"scan_select,{plate['offsetX']},{plate['offsetY']},{plate['colDist']},{plate['rowDist']},{plate['rows']},{plate['cols']},{wait_interval},{start_well},{end_well}\n"

    # Open serial communication
    ser = serial.Serial(port, 9600, timeout=2)
    time.sleep(0.5)
    ser.write(command.encode())

    print(f"Sent: {command.strip()}")
    print("Listening to Arduino")

    time.sleep(0.1)

    # Listen for responses from Arduino
    while True:
        line = ser.readline().decode().strip()
        if line:
            print("Arduino:", line, end='\r')  # Overwrites the current line
            if line.startswith("Well: "):
                well = line.replace("Well: ", "")
                well_queue.put(well)
                print(f"Well: {well} ", end='', flush=True)  # Append to the same line
                
                # Collect and save the measurement for the current well
                collect_ref(device, well)

            elif "Scan completed" in line:
                break

    ser.close()

def load_spectrum_from_csv(filepath):
    # Load spectrum data from a CSV file
    try:
        data = np.genfromtxt(filepath, delimiter=',', skip_header=1)
        return data[:, 1]  # return only the intensity (second column)
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None

# Function for collecting a single measurement and calculating the calibrated spectrum
def collect_single(device, well):
    print(f"Collecting measurement for well {well}...")

    data_folder = 'data'
    calib_folder = 'dark_ref_data'
    os.makedirs(data_folder, exist_ok=True)

    dark_path = os.path.join(calib_folder, f"{well}dark.csv")
    ref_path = os.path.join(calib_folder, f"{well}ref.csv")

    dark = load_spectrum_from_csv(dark_path)
    ref = load_spectrum_from_csv(ref_path)

    if dark is None or ref is None:
        print(f"Missing dark or reference file for well {well}. Skipping.")
        return {}

    try:
        # Measure spectrum
        spec = np.array(device.get_formatted_spectrum())
        wl = device.get_wavelengths()

        # Calibrate
        with np.errstate(divide='ignore', invalid='ignore'):
            calibrated = (spec - dark) / (ref - dark)
            calibrated = np.nan_to_num(calibrated, nan=0.0, posinf=0.0, neginf=0.0)

        # Save calibrated spectrum
        filename = os.path.join(data_folder, f"{well}.csv")
        with open(filename, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["Wavelength (nm)", "Reflectance"])
            for w, r in zip(wl, calibrated):
                writer.writerow([w, r])

        print(f"Saved calibrated data to {filename}")
        return {well: calibrated}

    except OceanDirectError as e:
        print(f"Measurement failed at {well}: {e.get_error_details()}")
        return {}


# Kamome and OceanDirect scan the selected range for actual data collection. Similar to the scan_ref and scan_dark functions, 
# the function first sends a serial command to Kamome to start the motor movements. The data is then captured once Kamome reports
# back its current well position. The spectra are saved into the 'data' folder. 
def scan_select(port, plate_type_name, wait_interval, start_well, end_well, device):
    import queue
    input("Starting data collection Press Enter when ready")
    
    well_queue = queue.Queue()

    plate = plate_types.get(plate_type_name)
    if not plate:
        print("Invalid plate type")
        return None, None

    # Prepare the scan command
    command = f"scan_select,{plate['offsetX']},{plate['offsetY']},{plate['colDist']},{plate['rowDist']},{plate['rows']},{plate['cols']},{wait_interval},{start_well},{end_well}\n"

    # Open serial communication
    ser = serial.Serial(port, 9600, timeout=2)
    time.sleep(0.5)
    ser.write(command.encode())

    print(f"Sent: {command.strip()}")
    print("Listening to Arduino")

    time.sleep(0.1)

    # Listen for responses from Arduino
    while True:
        line = ser.readline().decode().strip()
        if line:
            print("Arduino:", line, end='\r')  # Overwrites the current line
            if line.startswith("Well: "):
                well = line.replace("Well: ", "")
                well_queue.put(well)
                print(f"Well: {well} ", end='', flush=True)  # Append to the same line
                
                # Collect and save the measurement for the current well
                collect_single(device, well)

            elif "Scan completed" in line:
                break

    ser.close()

def calculate_selected_samples(start, end, plate):
    start_row = ord(start[0].upper()) - ord('A') + 1
    end_row = ord(end[0].upper()) - ord('A') + 1
    start_col = int(start[1:])
    end_col = int(end[1:])
    row_count = abs(end_row - start_row) + 1
    col_count = abs(end_col - start_col) + 1
    return row_count * col_count

# Function to completely close device instances
def graceful_shutdown(device, od):
    # Close device and release resources
    try:
        device.close_device()
        print("Spectrometer device closed.")
    except Exception as e:
        print(f"Error closing spectrometer: {e}")

    # Delete device object
    try:
        del device
        print("Device object deleted.")
    except Exception as e:
        print(f"Error deleting device object: {e}")

    # Shutdown OceanDirect API and release resources
    try:
        od.shutdown()
        print("OceanDirect API shutdown complete.")
    except Exception as e:
        print(f"Error during API shutdown: {e}")

# Setup OceanDirect and open the spectrometer

In [ ]:
# Setup device
import sys
sys.path.append(r"Projects\Kamome_OT2_OceanDirect\Reflectance\OceanDirect SDK\Python")

try:
    from oceandirect.od_logger import od_logger
    from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError
    print(">>> Setup Complete！")
    print("Python path:", sys.executable)
except Exception as e:
    print(f">>> Setup Failed: {e}")
    
od = OceanDirectAPI()
device_count = od.find_usb_devices()
device_ids = od.get_device_ids()

if device_count == 0:
    raise RuntimeError("No spectrometer found.")

print(f"Detected {device_count} device(s): {device_ids}")
device = od.open_device(device_ids[0])
device.set_scans_to_average(5)
device.set_integration_time(20000)  # 20 ms

# Set parameters for scanning

In [ ]:
port = "COM3"  # Update these as needed
plate_type = "Corning_96"
interval_ms = 1500
start_well = "A8" 
end_well = "D8"

# Calibrate the spectrometer

In [ ]:
scan_dark(port, plate_type, "1500", "A1", "H12", device)   # Change the start and end wells to match the plate type. e.g. Corning 24: start = A1, end = D6
scan_ref(port, plate_type, "1500", "A1", "H12", device)

# Scan samples

In [ ]:
spectra = scan_select(port, plate_type, interval_ms, start_well, end_well, device)

# Shutdown the spectrometer instance 
Make sure to run this everytime the notebook is interrupted above

In [ ]:
graceful_shutdown(device, od)